In [2]:
from bs4 import BeautifulSoup
import requests


url = 'https://www.theguardian.com/world'
headers = {
    'Accept': '*/*', 
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.167 YaBrowser/22.7.3.799 Yowser/2.5 Safari/537.36'
}
req = requests.get(url, headers=headers)
src = req.text
soup = BeautifulSoup(src, 'lxml')
items = []
allHrefs = soup.find_all(class_="dcr-lv2v9o")
allText = soup.find_all(class_="show-underline dcr-adlhb4")
allImage = soup.find_all(class_="dcr-evn1e9")
max_len = max(len(allHrefs), len(allText))
for i in range(max_len):
    try:
        itemText = allText[i].text
        itemHref = allHrefs[i].get('href')
        itemImage = allHrefs[i].get('src')
        fullHref = "https://www.theguardian.com" + str(itemHref)
        items.append(
            {
                "title": itemText,
                "href": fullHref,
                "image": itemImage
            }
        )
    except Exception:
        print(Exception)

items


AttributeError: ResultSet object has no attribute 'get'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [1]:
allImage[0].get('src')

NameError: name 'allImage' is not defined

In [ ]:
import pandas as pd
import spacy
import numpy as np
import pickle
import geonamescache

nlp = spacy.load('en_core_web_lg')
gc = geonamescache.GeonamesCache()
   

df = pd.DataFrame(items)
preprocessing_text = []
for text in df['title']:
    doc = nlp(text)
    new_text = []
    for token in doc:
        if not token.is_stop and not token.is_space and not token.is_punct:
            new_text.append(token.lemma_)
    preprocessing_text.append(' '.join(new_text))
df['preprocessing_text'] = preprocessing_text
df['vector'] = df['preprocessing_text'].apply(
    lambda text: nlp(text).vector)
preprocessing_text = np.stack(df.vector)
with open('app/data_collection/model_pickle', 'rb') as file:
    rfc = pickle.load(file)
y_pred = rfc.predict(preprocessing_text)
need_list = []
url_list = []
i = 0
for item in df['title']:
    if y_pred[i] == 1:
        print(item, y_pred[i])
        need_list.append(item)
    i += 1
i = 0
for it in df['href']:
    if y_pred[i] == 1:
        print(item, y_pred[i])
        url_list.append(it)
    i += 1
items = []
for i in range(len(need_list)):
    items.append(
        {
            "title": need_list[i],
            "id": i,
            "href": url_list[i]
        }
    )
countries = gc.get_countries()
cities = gc.get_cities()
def gen_dict_extract(var, key):
    if isinstance(var, dict):
        for k, v in var.items():
            if k == key:
                yield v
            if isinstance(v, (dict, list)):
                yield from gen_dict_extract(v, key)
    elif isinstance(var, list):
        for d in var:
            yield from gen_dict_extract(d, key)
countries = [*gen_dict_extract(countries, 'name')]
cities = [*gen_dict_extract(cities, 'name')]
info = []
for item in range(len(need_list)):
    doc = nlp(need_list[item])
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text in countries:
                info.append(
                    {
                        "country": ent.text,
                        "text": doc.text
                    }
                )
            elif ent.text not in cities:
                if ent.text == "US":
                    info.append(
                    {
                        "country": "United States of America",
                        "text": doc.text
                    }
                )
                elif ent.text == "UK":
                    info.append(
                    {
                        "country": "United Kingdom",
                        "text": doc.text
                    }
                )
                else:
                    info.append(
                        {
                            "country": ent.text,
                            "text": doc.text
                        }
                    )


: 